<a href="https://colab.research.google.com/github/Ronan-Castro/Pokedex-printable-for-binder/blob/main/Gera_Pokdex_Printer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# intervalo de pokémons desejado
inicio = 1
fim = 493

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!pip install reportlab pillow pandas


In [9]:
import requests
from reportlab.lib.pagesizes import A4, landscape
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
from reportlab.lib.colors import HexColor
from io import BytesIO

# === CONFIGURAÇÕES ===
OUTPUT_PDF = "/content/drive/MyDrive/Pokedex_API.pdf"


# cores e traduções de tipo
TIPO_CORES = {
    "Planta": "#78C850", "Venenoso": "#A040A0", "Fogo": "#F08030", "Água": "#6890F0",
    "Inseto": "#A8B820", "Normal": "#A8A878", "Voador": "#A890F0", "Elétrico": "#F8D030",
    "Terrestre": "#E0C068", "Pedra": "#B8A038", "Psíquico": "#F85888", "Gelo": "#98D8D8",
    "Lutador": "#C03028", "Fantasma": "#705898", "Dragão": "#7038F8", "Aço": "#B8B8D0",
    "Fada": "#EE99AC", "Sombrio": "#705848"
}

TIPO_TRAD = {
    "grass": "Planta", "poison": "Venenoso", "fire": "Fogo", "water": "Água",
    "bug": "Inseto", "normal": "Normal", "flying": "Voador", "electric": "Elétrico",
    "ground": "Terrestre", "rock": "Pedra", "psychic": "Psíquico", "ice": "Gelo",
    "fighting": "Lutador", "ghost": "Fantasma", "dragon": "Dragão", "steel": "Aço",
    "fairy": "Fada", "dark": "Sombrio"
}

# === FUNÇÃO PARA PEGAR DADOS ===
def pegar_pokemon(numero):
    url = f"https://pokeapi.co/api/v2/pokemon/{numero}"
    r = requests.get(url)
    if r.status_code != 200:
        return None
    data = r.json()
    nome = data["name"].capitalize()
    tipos = [TIPO_TRAD.get(t["type"]["name"], t["type"]["name"]) for t in data["types"]]
    tipo_txt = "/".join(tipos)
    sprite_url = data["sprites"]["front_default"]
    sprite_img = None
    if sprite_url:
        sprite_img = BytesIO(requests.get(sprite_url).content)
    return {"numero": numero, "nome": nome, "tipo": tipo_txt, "sprite": sprite_img}

# === BAIXAR TODOS ===
pokemons = []
for n in range(inicio, fim + 1):
    p = pegar_pokemon(n)
    if p:
        pokemons.append(p)
print(f"✅ {len(pokemons)} Pokémon baixados")

# === GERAR PDF ===
c = canvas.Canvas(OUTPUT_PDF, pagesize=landscape(A4))
width, height = landscape(A4)

cols, rows = 5, 6
margin_x, margin_y = 40, 40
cell_w, cell_h = (width - 2*margin_x) / cols, (height - 2*margin_y) / rows

for idx, row in enumerate(pokemons):
    page_index = idx % (cols*rows)
    if page_index == 0 and idx != 0:
        c.showPage()
    x = margin_x + (page_index % cols) * cell_w
    y = height - margin_y - ((page_index // cols) + 1) * cell_h

    # sprite remoto
    if row["sprite"]:
        try:
            img = ImageReader(row["sprite"])
            iw, ih = img.getSize()
            max_sprite_w, max_sprite_h = 60, 60
            scale = min(max_sprite_w / iw, max_sprite_h / ih)
            c.drawImage(img, x + 15, y + cell_h - ih * scale - 8, iw * scale, ih * scale, mask='auto')
        except Exception:
            pass

    # número + nome
    c.setFont("Helvetica-Bold", 10)
    text_x = x + 75
    text_y = y + cell_h - 18
    c.setFillColor(HexColor("#000000"))
    c.drawString(text_x, text_y, f"#{row['numero']:03d} {row['nome']}")

    # tipos alinhados fixos
    tipos = [t.strip() for t in str(row['tipo']).split('/') if t.strip()]
    spacing, box_h, box_w = 6, 13, 40
    box_y = text_y - 22
    start_x = text_x

    if tipos:
        # primeira caixa
        t1 = tipos[0]
        c.setFillColor(HexColor(TIPO_CORES.get(t1, "#DDDDDD")))
        c.rect(start_x, box_y, box_w, box_h, fill=1, stroke=0)
        c.setFillColor(HexColor("#000000"))
        c.setFont("Helvetica", 8)
        c.drawString(start_x + 4, box_y + 3, t1)
    if len(tipos) > 1:
        t2 = tipos[1]
        bx2 = start_x + box_w + spacing
        c.setFillColor(HexColor(TIPO_CORES.get(t2, "#DDDDDD")))
        c.rect(bx2, box_y, box_w, box_h, fill=1, stroke=0)
        c.setFillColor(HexColor("#000000"))
        c.drawString(bx2 + 4, box_y + 3, t2)

c.save()
print(f"📘 PDF gerado com sucesso: {OUTPUT_PDF}")


✅ 6 Pokémon baixados
📘 PDF gerado com sucesso: /content/drive/MyDrive/Pokedex_API.pdf
